In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [ ]:
# Import a list of "about" links
about_links_df = pd.read_csv("/content/drive/MyDrive/Priceline/JintongYu/TA_about_links.csv")
about_links_list = about_links_df["TA_main_page"].tolist()

In [ ]:
# Define a user agent to qualify request
user_agent = ({"User-Agent":
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                    AppleWebKit/537.36 (KHTML, like Gecko) \
                    Chrome/90.0.4430.212 Safari/537.36",
               "Accept-Language": "en-US, en;q=0.5"})

In [ ]:
# Extract the "about" section for each city
from numpy import NAN
def extract_content(url, soup):
  try:
    about_title = soup.find('div', {'class':'biGQs _P fiohW mowmC EVnyE'}).text
    about_content = soup.find('div', {'class':'GYFPJ'}).text
    return{
        "url": url,
        "about_title": about_title,
        "about_content": about_content
    #     "about_title": soup.find('div', {'class':'biGQs _P fiohW mowmC EVnyE'}).text,
    #     "about_content": soup.find('div', {'class':'GYFPJ'}).text
    }
  except:
    return{
        "url": url,
        "about_title": NAN,
        "about_content": NAN
    }

results = []
for url in about_links_list:
  response = requests.get(url, headers = user_agent, params={"url": url})
  soup = BeautifulSoup(response.text, "html.parser")
  results.append(extract_content(url, soup))

In [ ]:
# Convert the dictionary to a data frame
about_df = pd.DataFrame.from_dict(results)
about_df.head()

,url,about_title,about_content
0,https://www.tripadvisor.com/Tourism-g187791-Ro...,About Rome,"All roads lead to Rome, so they say. Or maybe ..."
1,https://www.tripadvisor.com/Tourism-g187497-Ba...,"Perfect beaches, world-famous tapas, and Gaudí...",There’s nowhere in the world like Barcelona. F...
2,https://www.tripadvisor.com/Tourism-g293974-Is...,A spellbinding city where cultures collide,"Over the centuries, many cultures have added t..."
3,https://www.tripadvisor.com/Tourism-g187849-Mi...,About Milan,If you skip Milan in favor of Italy’s more pop...
4,https://www.tripadvisor.com/Tourism-g187895-Fl...,"A wonderland of art and architecture, all fuel...",Florence makes art-lovers' hearts beat double ...


In [ ]:
# Check whether there is no "about" section for a city's TripAdvisor main page
about_df.describe(include='all')

,url,about_title,about_content
count,180,174,174
unique,180,174,174
top,https://www.tripadvisor.com/Tourism-g187791-Ro...,About Rome,"All roads lead to Rome, so they say. Or maybe ..."
freq,1,1,1


In [ ]:
# Concate the "about" dataframe with the original city list
about_df_concat = pd.concat([about_links_df.iloc[:, 0:5], about_df], axis=1)
about_df_concat.head()

,region_name,country_code,province_name,city_name,region_city_rank,url,about_title,about_content
0,SOUTHERN EUROPE,IT,LAZIO,ROME,1,https://www.tripadvisor.com/Tourism-g187791-Ro...,About Rome,"All roads lead to Rome, so they say. Or maybe ..."
1,SOUTHERN EUROPE,SP,CATALONIA,BARCELONA,2,https://www.tripadvisor.com/Tourism-g187497-Ba...,"Perfect beaches, world-famous tapas, and Gaudí...",There’s nowhere in the world like Barcelona. F...
2,SOUTHERN EUROPE,TU,ISTANBUL,ISTANBUL,3,https://www.tripadvisor.com/Tourism-g293974-Is...,A spellbinding city where cultures collide,"Over the centuries, many cultures have added t..."
3,SOUTHERN EUROPE,IT,LOMBARDIA,MILAN,4,https://www.tripadvisor.com/Tourism-g187849-Mi...,About Milan,If you skip Milan in favor of Italy’s more pop...
4,SOUTHERN EUROPE,IT,TOSCANA (TUSCANY),FLORENCE,5,https://www.tripadvisor.com/Tourism-g187895-Fl...,"A wonderland of art and architecture, all fuel...",Florence makes art-lovers' hearts beat double ...


In [ ]:
# List the cities w/o the "about" section
about_df_concat[about_df_concat['about_title'].isnull()]

,region_name,country_code,province_name,city_name,region_city_rank,url,about_title,about_content
14,CARIBBEAN,JM,NaN,HANOVER,5,https://www.tripadvisor.com/Tourism-g1448458-H...,NaN,NaN
52,UNITED STATES,US,FLORIDA,LAKE BUENA VISTA,13,https://www.tripadvisor.com/Tourism-g23869903-...,NaN,NaN
98,UNITED STATES,US,MINNESOTA,BLOOMINGTON,59,https://www.tripadvisor.com/Tourism-g42881-Blo...,NaN,NaN
101,UNITED STATES,US,CONNECTICUT,UNCASVILLE,62,https://www.tripadvisor.com/Tourism-g33952-Unc...,NaN,NaN
124,CENTRAL/SOUTH AMERICA,CS,NaN,GUANACASTE,10,https://www.tripadvisor.com/Tourism-g309235-Pr...,NaN,NaN
138,AFRICA,NI,NaN,LAGOS,4,https://www.tripadvisor.com/Tourism-g304026-La...,NaN,NaN


In [ ]:
# Save to a CSV file
about_df_concat.to_csv("about.csv")